# Лабораторная работа 5
# 10 группа ММАД, 5 вариант
# Квиткевич Александр

# Методом Монте-Карло найти решение СЛАУ:

\begin{equation}
B * x = g
\end{equation}

\begin{equation}
B = \begin{bmatrix}
0.8 & 0.2 & 0.3\\
-0.2 & 0.5 & -0.3\\
0.4 & 0.2 & 1.3
\end{bmatrix}, 
g = \begin{bmatrix}
3\\
-1\\
-1
\end{bmatrix}
\end{equation}

\begin{equation}
Solution: \tilde{x} = \begin{bmatrix}
4.84732824\\
-1.29770992\\
-2.0610687
\end{bmatrix}
\end{equation}

In [1]:
import numpy as np
from scipy.linalg import eigvals, norm, solve
from tqdm import trange  # for interactive progress checking


np.random.seed(42)
B = np.array([[0.8, 0.2, 0.3], [-0.2, 0.5, -0.3], [0.4, 0.2, 1.3]])
g = np.array([3, -1, -1])

analytical_ans = solve(B, g)
print("Матрица B:")
print(B)
print("\nВектор g:")
print(g)
print("\nАналитическое решение:")
print(analytical_ans)

Матрица B:
[[ 0.8  0.2  0.3]
 [-0.2  0.5 -0.3]
 [ 0.4  0.2  1.3]]

Вектор g:
[ 3 -1 -1]

Аналитическое решение:
[ 4.84732824 -1.29770992 -2.0610687 ]


# Преобразуем систему к необходимому для итераций виду
\begin{equation}
x = A * x + f
\end{equation}

\begin{equation}
A = \begin{bmatrix}
0.2 & -0.2 & -0.3\\
0.4 & 0 & 0.6\\
-0.4 & -0.2 & -0.3
\end{bmatrix}, 
f = \begin{bmatrix}
3\\
-2\\
-1
\end{bmatrix}
\end{equation}

In [2]:
A = np.array([[0.2, -0.2, -0.3], [0.4, 0, 0.6], [-0.4, -0.2, -0.3]])
f = np.array([3, -2, -1])
print("Матрица A:")
print(A)
print("\nВектор f:")
print(f)

Матрица A:
[[ 0.2 -0.2 -0.3]
 [ 0.4  0.   0.6]
 [-0.4 -0.2 -0.3]]

Вектор f:
[ 3 -2 -1]


In [3]:
for eigval in eigvals(A):
    eigval_modulo = abs(eigval)
    assert eigval_modulo < 1, "Собственное значение больше 1 по модулю!"
    print(
        f"Собственное значение матрицы A: {np.round(eigval, 2)}, его модуль: {np.round(abs(eigval), 2)}"
    )
print(
    "\nВсе собственные значения меньше 1 по модулю, матрица B  удовлетворяет условиям метода Монте-Карло"
)

Собственное значение матрицы A: (0.37+0j), его модуль: 0.37
Собственное значение матрицы A: (-0.24+0.37j), его модуль: 0.44
Собственное значение матрицы A: (-0.24-0.37j), его модуль: 0.44

Все собственные значения меньше 1 по модулю, матрица B  удовлетворяет условиям метода Монте-Карло


In [4]:
n = 3  # размерность системы
N = 300  # длина цепи Маркова
m = n * 30000  # количество реализаций цепи Маркова

h = np.eye(n)
pi = np.ones_like(f) * (1.0 / n)
P = np.array(np.ones_like(A)) * (1 / n)

print("Матрица для вычислений x,y,z:")
print(h)
print("\nВектор вероятностей начальных состояний цепи Маркова:")
print(pi)
print("\nМатрица переходов:")
print(P)

Матрица для вычислений x,y,z:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Вектор вероятностей начальных состояний цепи Маркова:
[0.33333333 0.33333333 0.33333333]

Матрица переходов:
[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


In [5]:
ksi = np.zeros((m, n), dtype=float)
idxs = np.array(np.random.rand(m, N) // (1 / n), dtype=int)
Q = np.zeros((m, N, n), dtype=float)

for j in trange(m):

    idx = idxs[j][
        0
    ]  # idx shows which of x, y, z we will modify (start of each Markov chain)
    Q[j][0][idx] = 0 if not pi[idx] else h[idx, idx] / pi[idx]
    for k in range(1, N):
        idx_prev = idxs[j][k - 1]  # idx_prev is previous state of Markov chain
        idx_new = idxs[j][k]  # idx_new is new state of Markov chain
        Q[j][k][idx] = (
            0
            if not P[idx_prev][idx_new]
            else Q[j][k - 1][idx] * A[idx_prev][idx_new] / P[idx_prev][idx_new]
        )

    ksi[j] = np.dot(f[idxs[j]], Q[j])

monte_carlo_ans = ksi.mean(axis=0)
print("Решение по методу Монте-Карло:", monte_carlo_ans)

diff_norm = norm(analytical_ans - monte_carlo_ans)
print(
    "Норма разности аналитического и численного решения, полученного по методу Монте-Карло:",
    round(diff_norm, 2),
)

error = np.dot(B, monte_carlo_ans) - g
print("Ошибка ММК-решения:", error)

100%|██████████| 90000/90000 [01:10<00:00, 1269.09it/s]


Решение по методу Монте-Карло: [ 4.88586846 -1.2110391  -2.10115043]
Норма разности аналитического и численного решения, полученного по методу Монте-Карло: 0.1
Ошибка ММК-решения: [ 0.03614182  0.04765189 -0.019356  ]
